# Image / Mask transformers
Change image and mask information

In [161]:
import torchvision.transforms.functional as TF
import torch
torch.random.manual_seed(42)

class Resize:
    '''
    Resize image and mask
    Args:
        size - tulpe of int
        sample - {PIL image, PIL mask}
    '''
    def __init__(self, size):
        self.size = size

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        image = TF.resize(image, self.size)
        mask = TF.resize(mask, self.size)
    
        return {'image': image, 'mask': mask}
    
class ToTensor(object):
    '''
    Convert PIL image and mask in sample to Tensors.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        return {'image': TF.to_tensor(image),
                'mask': TF.to_tensor(mask)}
    
class RandAffine:
    '''
    Apply affine transformation on the image keeping image center invariant.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # generate parameters
        angle = torch.randint(-180,180, (1,1)).item()
        translate = (torch.randint(0,5,(1,1)).item(), 
                     torch.randint(0,5,(1,1)).item())
        scale = torch.div(torch.randint(50,200, (1,1)), 100.).item()
        shear = torch.randn(1).item()
        
        image = TF.affine(image, angle, translate, scale, shear)
        mask   = TF.affine(mask,  angle, translate, scale, shear)
        
        return {'image': image, 'mask': mask}

class HFlip:
    '''
    Horizontally flip the given PIL Image or torch Tensor.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        image = TF.hflip(image)
        mask   = TF.affine(mask)
        
        
        
        return {'image': image, 'mask': mask}
    

# Image transformers
Change only image information

In [ ]:
_lower_bound = 50
_upper_boubd = 150
class AdjBrightness:
    '''
    Adjust brightness of an Image.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # float random generator
        factor = torch.div(torch.randint(_lower_bound,_upper_boubd, (1,1)), 100.).item() 
        
        image = TF.adjust_brightness(image, factor)
        
        return {'image': image, 'mask': mask}
    
class AdjContrast:
    '''
    Adjust contrast of an Image.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # float random generator
        factor = torch.div(torch.randint(_lower_bound,_upper_boubd, (1,1)), 100.).item() 
        
        image = TF.adjust_contrast(image, factor)
        
        return {'image': image, 'mask': mask}
    
class AdjSaturation:
    '''
    Adjust saturation of an Image.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # float random generator
        factor = torch.div(torch.randint(_lower_bound,_upper_boubd, (1,1)), 100.).item() 
        
        image = TF.adjust_saturation(image, factor)
        
        return {'image': image, 'mask': mask}
    
class AdjHue:
    '''
    Adjust hue of an Image.
    Args:
        sample - {PIL image, PIL mask}
    '''
    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        
        # float random generator
        factor = torch.div(torch.randint(-50,50, (1,1)), 100.).item() 
        
        image = TF.adjust_hue(image, factor)
        
        return {'image': image, 'mask': mask}
    
    